In [ ]:
# Install necessary packages quietly
!pip install rdkit scikit-learn pandas numpy -q

# Import required libraries
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, matthews_corrcoef, balanced_accuracy_score)
import pandas as pd
import numpy as np
import random

# Seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load the classification data
df = pd.read_csv('top_30_features_from_mo.tsv', sep='\t')

# Handle missing values with imputation
imputer = SimpleImputer(strategy='mean')
df[df.select_dtypes(include=[float, int]).columns] = imputer.fit_transform(df.select_dtypes(include=[float, int]))

# Select numerical features and target variable
numerical_features = df.select_dtypes(include=[float, int]).columns.drop(['Label'])  # Adjust column names as needed
X = df[numerical_features].values
y = df['Label'].values

# Split data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y)

# Scale numerical features (only if necessary)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters from Grid Search
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Compute evaluation metrics
metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score": f1_score(y_test, y_pred),
    "ROC AUC": roc_auc_score(y_test, y_pred_proba),
    "MCC": matthews_corrcoef(y_test, y_pred),
}

print("\nTest Set Evaluation Metrics (Best Model):")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                             roc_auc_score, roc_curve, matthews_corrcoef, balanced_accuracy_score, precision_score, recall_score, f1_score)
from sklearn.impute import SimpleImputer
from PIL import Image

# Read data
df = pd.read_csv('top_30_features_from_mo.tsv', sep='\t')

# Handle missing values by imputing with the mean
imputer = SimpleImputer(strategy="mean")
X = df.drop(['Label', 'SMILES'], axis=1)
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Extract labels
y = df['Label'].values

# Initialize scaler and classifier parameters
scaler = StandardScaler()
classifier_params = dict(n_estimators=300, random_state=0, bootstrap=False,
                         max_depth=None, max_features='sqrt',
                         min_samples_leaf=1, min_samples_split=2)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to collect metrics per fold
acc_scores, auc_scores, mcc_scores, bal_acc_scores = [], [], [], []
precision_scores, recall_scores, f1_scores = [], [], []

# Store ROC curve points for each fold
fold_rocs = []

for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train classifier
    clf = RandomForestClassifier(**classifier_params)
    clf.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = clf.predict(X_test_scaled)
    y_pred_proba = clf.predict_proba(X_test_scaled)[:, 1]
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    mcc = matthews_corrcoef(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')
    
    acc_scores.append(acc)
    auc_scores.append(auc)
    mcc_scores.append(mcc)
    bal_acc_scores.append(bal_acc)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    print("Test Set Metrics:")
    print(f"  Accuracy (ACC): {acc:.4f}")
    print(f"  ROC AUC: {auc:.4f}")
    print(f"  MCC: {mcc:.4f}")
    print(f"  Balanced Accuracy: {bal_acc:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    
    # Save confusion matrix as image
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6, 4.5))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    classes = np.unique(y)
    ax.set(xticks=np.arange(len(classes)),
           yticks=np.arange(len(classes)),
           xticklabels=classes, yticklabels=classes,
           title=f'Fold {fold} Confusion Matrix',
           ylabel='True label',
           xlabel='Predicted label')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > cm.max() / 2. else "black")
    fig.tight_layout()

    # Save the image
    png_path = f'confusion_matrix_fold_{fold}.png'
    fig.savefig(png_path, dpi=600, format='png', pil_kwargs={"optimize": True, "quality": 95})
    plt.close(fig)
    
    # Save ROC curve data for combined plot
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    fold_rocs.append((fpr, tpr, auc, fold))

# Train final model on full data
print("\n=== Mean Model (Full Dataset) ===")
X_scaled = scaler.fit_transform(X)
clf_full = RandomForestClassifier(**classifier_params)
clf_full.fit(X_scaled, y)

y_pred_full = clf_full.predict(X_scaled)
y_pred_proba_full = clf_full.predict_proba(X_scaled)[:, 1]

acc_full = accuracy_score(y, y_pred_full)
auc_full = roc_auc_score(y, y_pred_proba_full)
mcc_full = matthews_corrcoef(y, y_pred_full)
bal_acc_full = balanced_accuracy_score(y, y_pred_full)
precision_full = precision_score(y, y_pred_full, average='binary')
recall_full = recall_score(y, y_pred_full, average='binary')
f1_full = f1_score(y, y_pred_full, average='binary')

# Save confusion matrix for mean model
cm = confusion_matrix(y, y_pred_full)
fig, ax = plt.subplots(figsize=(6, 4.5))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
classes = np.unique(y)
ax.set(xticks=np.arange(len(classes)),
       yticks=np.arange(len(classes)),
       xticklabels=classes, yticklabels=classes,
       title='Mean Model Confusion Matrix',
       ylabel='True label',
       xlabel='Predicted label')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > cm.max() / 2. else "black")
fig.tight_layout()

png_path = 'mean_model_confusion_matrix.png'
fig.savefig(png_path, dpi=600, format='png', pil_kwargs={"optimize": True, "quality": 95})
plt.close(fig)

# Plot combined ROC curves and save
fig, ax = plt.subplots(figsize=(8, 6))
for fpr, tpr, auc, fold in fold_rocs:
    ax.plot(fpr, tpr, linestyle=':', lw=2, label=f'Fold {fold} ROC (AUC = {auc:.4f})')

fpr_full, tpr_full, _ = roc_curve(y, y_pred_proba_full)
mean_auc = np.mean(auc_scores)  # Use mean AUC from CV folds here

ax.plot(fpr_full, tpr_full, color='blue', lw=2,
        label=f'Mean Model ROC (AUC = {mean_auc:.4f})', linestyle='-')
ax.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Chance')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves: 5-fold CV (dotted) & Mean Model (solid): RF')
ax.legend(loc='lower right')
ax.grid(alpha=0.3)
fig.tight_layout()
png_path = 'roc_curves.png'
fig.savefig(png_path, dpi=600, format='png', pil_kwargs={"optimize": True, "quality": 95})
plt.close(fig)




# Calculate means and standard deviations for metrics
metrics = {
    "Accuracy (ACC)": (np.mean(acc_scores), np.std(acc_scores)),
    "ROC AUC": (np.mean(auc_scores), np.std(auc_scores)),
    "MCC": (np.mean(mcc_scores), np.std(mcc_scores)),
    "Balanced Accuracy": (np.mean(bal_acc_scores), np.std(bal_acc_scores)),
    "Precision": (np.mean(precision_scores), np.std(precision_scores)),
    "Recall": (np.mean(recall_scores), np.std(recall_scores)),
    "F1 Score": (np.mean(f1_scores), np.std(f1_scores)),
}

# Print metrics with mean ± std for cross-validation
print("\n=== Cross-validation Metrics (Mean ± Std) ===")
for metric, (mean, std) in metrics.items():
    print(f"{metric}: {mean:.4f} ± {std:.4f}")
